In [ ]:
import json
from config import settings
import os


import polars as pl
from polars import col as c
from polars import selectors as cs
import networkx as nx

from typing_extensions import Union
import numpy as np
import scipy as sp
import patito as pt

# from distflow_algorithm import DistFlow
import time
import pandapower as pp
import pandapower.networks as pn


from data_connector import pandapower_to_distflow
from general_function import duckdb_to_dict, dict_to_duckdb, pl_to_dict
from networkx_function import (
    get_all_edge_data,
    generate_shortest_path_length_matrix,
    generate_bfs_tree_with_edge_data,
    generate_nx_edge,
    generate_tree_graph_from_edge_data,
)
import graphblas as gb
from utility.parser_utility import duckdb_to_changes_schema

from twindigrid_sql.entries.equipment_class import (
    TRANSFORMER,
    BRANCH,
    SWITCH,
    EXTERNAL_NETWORK,
)

from distflow_schema import DistFlowSchema

from state_estimation.matrix_generators import generate_full_jacobian_matrix

# Useless outside jupiternotebook because in settings.py a line that changes the directory to src for ipynb
os.chdir(os.getcwd().replace("/src", ""))

## Import Schema from duckdb

In [1]:
file_names: dict[str, str] = json.load(open(settings.INPUT_FILE_NAMES))
changes_schema = duckdb_to_changes_schema(file_path=file_names["duckdb_output"])
list(changes_schema.__dict__.keys())

NameError: name 'json' is not defined

## Create resource data

In [ ]:
resource_data = changes_schema.resource.filter(  ##TODO is_in()
    (c("concrete_class") == BRANCH)
    | (c("concrete_class") == SWITCH)
    | (c("concrete_class") == TRANSFORMER)
    | (c("concrete_class") == EXTERNAL_NETWORK)
).select(
    c("uuid"), c("dso_code").alias("element_id"), c("concrete_class").alias("type")
)

## Add row index to connectivity_node to give the number of node
## Join connectivity_node with connectivity to get the eq_fk
## Create the connectivity_node dictionnary with side+eq_fk as key and node index as value
cn_mapping: dict[float, str] = pl_to_dict(
    changes_schema.connectivity_node.with_row_index()
    .join(changes_schema.connectivity, left_on="uuid", right_on="cn_fk", how="left")
    .select((c("side") + c("eq_fk")).alias("eq_fk_side"), c("index"))
)

## Add node from and node to for each edge with side+eq_fk
resource_data = resource_data.with_columns(
    ("t1" + c("uuid"))
    .replace_strict(cn_mapping, default=None)
    .alias(
        "u_of_edge"
    ),  # Replace side+eq_fk with node number from connectivity for equipment
    ("t2" + c("uuid")).replace_strict(cn_mapping, default=None).alias("v_of_edge"),
)
# Add branch parameter
branch = changes_schema.branch_parameter_event[["uuid", "eq_fk", "r", "x", "b", "g"]]

# Add branch parameter to line_data
resource_data = resource_data.join(
    branch, left_on="uuid", right_on="eq_fk", how="left"
).drop("uuid_right")
## Add n_tranfo to 1, useless, because automatically added by add_table from class DistFlowSchema and by default value is 1
# resource_data = resource_data.with_columns(pl.lit(1).alias("n_tranfo"))

## Search slack node id for DisFlowSchema
slack_node_id = resource_data.filter(c("type") == EXTERNAL_NETWORK)["u_of_edge"].item()

## Remove the external network from the resource_data
resource_data = resource_data.filter(c("type") != EXTERNAL_NETWORK)

## Transform resource_data in pu

In [ ]:
## Transfo in pu
s_base = 1e6  # VA -> 1 MVA for distribution grid

u_b = changes_schema.base_voltage["nominal_voltage"].item()  # V
i_b = s_base / (3**0.5 * u_b)  # A
z_b = u_b**2 / s_base  # Ohm
b_b = 1 / z_b  # S


pu_base_changes_schema = {
    "U_b": u_b,
    "I_b": i_b,
    "Z_b": z_b,
    "B_b": b_b,
    "S_base": s_base,
}

resource_data_pu = resource_data.with_columns(
    (c("g") * pu_base_changes_schema["Z_b"]).alias("g_pu"),
    (c("r") / pu_base_changes_schema["Z_b"]).alias("r_pu"),
    (c("x") / pu_base_changes_schema["Z_b"]).alias("x_pu"),
    (c("b") / pu_base_changes_schema["B_b"]).alias("b_pu"),
).drop(["r", "x", "b", "g"])

## Check if B is negativ for branch and positiv for trafo

In [ ]:
## If type is branch, then b_pu is negative, otherwise positive (trafo and switch)
resource_data_pu = resource_data_pu.with_columns(
    pl.when(c("type") == BRANCH)
    .then(c("b_pu").abs().neg())
    .otherwise(c("b_pu").abs())
    .alias("b_pu")
)

## Instanciation of distflowschema

In [ ]:
##TODO add n_trafo for the transformer
## Create the DistFlowSchema instance
distflow_schema = DistFlowSchema()
## Add edge_data table to the DistFlowSchema instance
distflow_schema = distflow_schema.add_table(edge_data=resource_data_pu)

In [ ]:
distflow_schema.edge_data

u_of_edge,v_of_edge,r_pu,x_pu,b_pu,g_pu,n_transfo,type
i32,i32,f64,f64,f64,f64,f64,str
1,0,0.019375,0.011297,-7.9168e-7,0.0,1.0,"""branch"""
2,0,0.204688,0.114625,-0.000009,0.0,1.0,"""branch"""
4,1,0.303,0.00911,-2.2696e-7,0.0,1.0,"""branch"""
5,1,0.00775,0.004519,-3.1667e-7,0.0,1.0,"""branch"""
6,2,0.041406,0.0231875,-0.000002,0.0,1.0,"""branch"""
…,…,…,…,…,…,…,…
55,54,0.005425,0.003163,-2.2167e-7,0.0,1.0,"""branch"""
56,54,0.3219375,0.009679,-2.4114e-7,0.0,1.0,"""branch"""
57,55,0.2461875,0.007402,-1.8440e-7,0.0,1.0,"""branch"""


In [ ]:
generate_full_jacobian_matrix(distflow_schema, slack_node_id)

"M_23"     nvals  nrows  ncols  dtype     format
gb.Matrix    558     58     58   FP64  csr (iso)
------------------------------------------------
     0    1    2    3    4    5    6    7    8    9    10   11   12   13   14  \
0   1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   
1        1.0            1.0  1.0                      1.0  1.0                  
2             1.0                 1.0  1.0  1.0                 1.0  1.0        
3                  1.0                           1.0                      1.0   
4                       1.0                                                     
5                            1.0                      1.0  1.0                  
6                                 1.0                           1.0  1.0        
7                                      1.0                                      
8                                           1.0                                 
9                                          

array([[ 1.        ,  0.        ,  0.        , ...,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        , ...,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.        , ...,  0.        ,  0.        ,  0.        ],
       ...,
       [-0.03875   ,  0.        ,  0.        , ..., -0.25252625, -0.2331675 ,  1.        ],
       [-0.03875   ,  0.        ,  0.        , ..., -0.2331675 , -0.2542975 ,  1.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,  0.        ,  1.        ]])

## Correct jacobian matrix

Add upflow and downflow

In [ ]:
edge_data: pt.DataFrame = distflow_schema.edge_data
if edge_data.is_empty():
    raise ValueError("edge_data is empty")
# Check if there is no parallel edges (nx.Graph does not support parallel edges instead of nx.MultiGraph)
if not edge_data.filter(pl.struct("u_of_edge", "v_of_edge").is_duplicated()).is_empty():
    raise ValueError("Edges in parallel in edge_data")

# check if the slack node is in the grid
if edge_data.filter(
    pl.any_horizontal(c("u_of_edge", "v_of_edge") == slack_node_id)
).is_empty():
    raise ValueError("The slack node is not in the grid")
# Create nx_tree from line data
nx_grid: nx.Graph = nx.Graph()
_ = edge_data.with_columns(pl.struct(pl.all()).pipe(generate_nx_edge, nx_graph=nx_grid))
# Check if the grid is a connected tree
if not nx.is_tree(nx_grid):
    raise ValueError("The grid is not a tree")
elif not nx.is_connected(nx_grid):
    raise ValueError("The grid is not connected")

nx_tree_grid: nx.DiGraph = generate_bfs_tree_with_edge_data(nx_grid, slack_node_id)

# remove slack node from list of nodes
nodes = np.array(nx_tree_grid.nodes())
nodes = nodes[nodes != slack_node_id]

n_nodes: int = nx_tree_grid.number_of_nodes()
n_edges: int = nx_tree_grid.number_of_edges()

descendent_matrix = generate_shortest_path_length_matrix(
    nx_grid=nx_tree_grid, forced_weight=1
)
# print(descendent_matrix)
# Create edge data graphblas vector
g_pu = gb.Vector.from_dense(edge_data["g_pu"])  # type: ignore
b_pu = gb.Vector.from_dense(edge_data["b_pu"])  # type: ignore
##TODO add n_trafo for the transformer with square value
n_transfo_sqr = gb.Vector.from_dense(edge_data["n_transfo"])  # type: ignore

# Matrix values represent the sum of longitudinal resistances (or reactance) of edges in the path connecting the slack node to the
# lowest common ancestor of i ang j (row and column index node).
coords, ancestor = list(zip(*nx.all_pairs_lowest_common_ancestor(nx_tree_grid)))
x, y = list(zip(*coords))

r_mapping = nx.shortest_path_length(nx_tree_grid, source=slack_node_id, weight="r_pu")
r_val = list(map(lambda node_id: -2 * r_mapping[node_id], ancestor))

vn_pload_gb = gb.Matrix.from_coo(x, y, r_val, nrows=n_nodes, ncols=n_nodes).select("!=", 0)[nodes, nodes]  # type: ignore
vn_pload = (
    gb.select.offdiag(vn_pload_gb)
    .T.ewise_add(  # type: ignore
        vn_pload_gb
    )  # type: ignore
    .to_dense(fill_value=0.0)  # type: ignore
)

x_mapping = nx.shortest_path_length(nx_tree_grid, source=slack_node_id, weight="x_pu")
x_val = list(map(lambda node_id: -2 * x_mapping[node_id], ancestor))

vn_qload_gb = gb.Matrix.from_coo(x, y, x_val, nrows=n_nodes, ncols=n_nodes).select("!=", 0)[nodes, nodes]  # type: ignore
vn_qload = (
    gb.select.offdiag(vn_qload_gb)
    .T.ewise_add(  # type: ignore
        vn_qload_gb
    )  # type: ignore
    .to_dense(fill_value=0.0)
)

# Matrix value is 1 if j (column index node) is downstream i (row index node) 0 otherwise
# pflow_pload = descendent_matrix[nodes, nodes].to_dense(fill_value=0.0)
# qflow_qload = descendent_matrix[nodes, nodes].to_dense(fill_value=0.0)

# TODO Check if we add or not half of upstream node
# Matrix value is the sum of transverse susceptance (or conductance) of downstream edge (branch or transformer)
# pflow_v0 = (
#     gb.select.offdiag(descendent_matrix)[nodes, nodes]  # type: ignore
#     .ewise_mult(g_pu)
#     .reduce_rowwise(gb.monoid.plus)  # type: ignore
#     .to_dense(fill_value=0.0)  # type: ignore
# ).reshape(-1, 1)

# qflow_v0 = (
#     gb.select.offdiag(descendent_matrix)[nodes, nodes]  # type: ignore
#     .ewise_mult(b_pu)  # type: ignore
#     .reduce_rowwise(gb.monoid.plus)  # type: ignore
#     .to_dense(fill_value=0.0)
# ).reshape(
#     -1, 1
# )  # type: ignore

# Matrix value is the multiplication of transformer ratio found in upstream edge
# (for switch and branch, n_transfo == 1)
## TODO add sqr to name
vn_v0 = (
    descendent_matrix.T[nodes, nodes]  # type: ignore
    .ewise_mult(n_transfo)  # type: ignore
    .reduce_rowwise(gb.monoid.times)  # type: ignore
    .to_dense(fill_value=0.0)
).reshape(
    -1, 1
)  # type: ignore

# Simple matrix
sload_sload = np.eye(2 * n_edges)
## TODO add sqr to name
sload_v0 = np.zeros([2 * n_edges, 1])  # type: ignore

# pflow_qload = np.zeros([n_edges, n_edges])  # type: ignore
# qflow_pload = np.zeros([n_edges, n_edges])  # type: ignore

## TODO add sqr to name for the two following matrix
v0_sload = np.zeros([1, 2 * n_edges])
v0_v0 = np.ones([1, 1])

In [ ]:
# Matrix value is 1 if j (column index node) is downstream i (row index node) 0 otherwise
pflow_pload = descendent_matrix[nodes, nodes].to_dense(fill_value=0.0)
qflow_qload = descendent_matrix[nodes, nodes].to_dense(fill_value=0.0)

## TODO same but negativ for down
# Matrix value is -1 if j (column index node) is upstream i (row index node) 0 otherwise
pflow_pload
qflow_qload

# TODO Check if we add or not half of upstream node
# Matrix value is the sum of transverse susceptance (or conductance) of downstream edge (branch or transformer)
##TODO up with diag branch i inside /// down without diag and negativ
pflow_v0_down = -(
    gb.select.offdiag(descendent_matrix)[nodes, nodes]  # type: ignore
    .ewise_mult(g_pu)
    .reduce_rowwise(gb.monoid.plus)  # type: ignore
    .to_dense(fill_value=0.0)  # type: ignore
).reshape(-1, 1)

qflow_v0 = (
    gb.select.offdiag(descendent_matrix)[nodes, nodes]  # type: ignore
    .ewise_mult(b_pu)  # type: ignore
    .reduce_rowwise(gb.monoid.plus)  # type: ignore
    .to_dense(fill_value=0.0)
).reshape(
    -1, 1
)  # type: ignore

pflow_qload = np.zeros([n_edges, n_edges])  # type: ignore
qflow_pload = np.zeros([n_edges, n_edges])  # type: ignore

In [ ]:
edge_data["n_transfo"].pow(2)

n_transfo
f64
1.0
1.0
1.0
1.0
1.0
…
1.0
1.0
1.0


In [ ]:
nx_tree_grid.edges()

OutEdgeView([(0, 1), (0, 2), (0, 3), (1, 4), (1, 5), (2, 6), (2, 7), (2, 8), (3, 9), (5, 10), (5, 11), (6, 12), (6, 13), (9, 14), (11, 15), (11, 16), (14, 17), (14, 18), (14, 19), (16, 20), (16, 21), (17, 22), (17, 23), (21, 24), (21, 25), (25, 26), (25, 27), (27, 28), (27, 29), (29, 30), (29, 31), (30, 32), (31, 33), (31, 34), (31, 35), (32, 36), (35, 37), (35, 38), (38, 39), (38, 40), (38, 41), (41, 42), (41, 43), (43, 44), (43, 45), (45, 46), (45, 47), (47, 48), (47, 49), (48, 50), (49, 51), (49, 52), (52, 53), (52, 54), (54, 55), (54, 56), (55, 57)])

In [ ]:
tewst = nx.adjacency_matrix(nx_tree_grid).toarray()

In [ ]:
(
    gb.select.offdiag(descendent_matrix)[nodes, nodes].ewise_mult(b_pu)  #  type: ignore
)  # type: ignore

gb.MatrixExpression                            nrows  ncols  dtype
M_231.ewise_mult(v_45, op=binary.times[FP64])     57     57   FP64

"Result"   nvals  nrows  ncols  dtype
gb.Matrix    443     57     57   FP64
-------------------------------------
   0  1  2    3         4         5    6    7    8    9         10      11  \
0           -0.0 -0.000002                          -0.0 -0.000001           
1                          -0.000001 -0.0 -0.0                     -0.5885   
2                                              -0.0                          
3                                                                            
4                                                   -0.0 -0.000001           
5                                                                  -0.5885   
6                                                                            
7                                                                            
8                                                                            
9                                                                            
10                                                                           
11                                                                           
12                                                                           
13                                                                           
14                                                                           
15                                                                           
16                                                                           
17                                                                           
18                                                                           
19                                                                           
20                                                                           
21                                                                           
22                                                                           
23                                                                           
24                                                                           
25                                                                           
26                                                                           
27                                                                           
28                                                                           
29                                                                           
30                                                                           
31                                                                           
32                                                                           
33                                                                           
34                                                                           
35                                                                           
36                                                                           
37                                                                           
38                                                                           
39                                                                           
40                                                                           
41                                                                           
42                                                                           
43                                                                           
44                                                                           
45                                                                           
46                                                                           
47     

In [ ]:
# Matrix concatenation
h = np.concatenate(
    [
        np.concatenate([sload_sload, sload_v0], axis=1),
        np.concatenate([pflow_pload, pflow_qload, pflow_v0], axis=1),
        np.concatenate([qflow_pload, qflow_qload, qflow_v0], axis=1),
        np.concatenate([vn_pload, vn_qload, vn_v0], axis=1),
        np.concatenate([v0_sload, v0_v0], axis=1),
    ],
    axis=0,
)

## To be deleted

NameError: name 'DistFlow' is not defined

In [ ]:
net: pp.pandapowerNet = pp.from_pickle("data/input_grid/synthesized_grid.p")
net.trafo["i0_percent"] = 2
tic = time.time()
pp.runpp(net)
print("f:", time.time() - tic)